In [1]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
from sklearn.metrics.pairwise import linear_kernel
import plotly.graph_objs as go
from chart_studio.plotly import plot, iplot
import cufflinks as cf
cf.go_offline(connected=True)

In [2]:
df = pd.read_csv('./data/preprocessing/preprocessed/new_tokenized.csv',index_col=0)
df.head()

,연도,crp_nm,rpt_nm,rcp_no,rcp_dt,str,str_tokens,str_nouns,str_ex_josa
0,2019,삼양홀딩스,사업보고서 (2018.12),20190401004413,20190401,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항당사가 본 경영진단의견서...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
1,2018,삼양홀딩스,사업보고서 (2017.12),20180402004217,20180402,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항당사가 본 경영진단의견서...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
2,2017,삼양홀딩스,사업보고서 (2016.12),20170331003683,20170331,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
3,2016,삼양홀딩스,사업보고서 (2015.12),20160330002684,20160330,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
4,2015,삼양홀딩스,사업보고서 (2014.12),20150331002399,20150331,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...


#### 사업보고서 fillings_month 체크

In [3]:
rcp_month = [int(str(x)[4:6]) for x in df.rcp_dt]
dic_ = {i:rcp_month.count(i) for i in set(rcp_month)}
pd.DataFrame(index=dic_.keys(),data= dic_.values()).iplot(kind='bar',title='월별 공시문서수',xTitle='month',yTitle='count')

## wordcount

In [4]:
def word_count(ls):  #document의 word count
    ls = ls.split()
    counts = dict()
    for word in ls:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
            
    return counts

In [5]:
def total_word_count(srs): #total document의 word count
    ls = [x.split() for x in srs]
    counts = dict()
    for doc in ls:
        for word in doc:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
    return counts

In [6]:
tokens_dic = total_word_count(df.str_tokens) #전체토큰
nouns_dic = total_word_count(df.str_nouns) #명사만
ex_josa_dic = total_word_count(df.str_ex_josa) #조사,어미 제외
tfd_tokens = pd.DataFrame(tokens_dic.items(),columns = ['단어','빈도']).sort_values('빈도',ascending=False).reset_index(drop=True)
tfd_nouns = pd.DataFrame(nouns_dic.items(),columns = ['단어','빈도']).sort_values('빈도',ascending=False).reset_index(drop=True)
tfd_ex_josa = pd.DataFrame(ex_josa_dic.items(),columns = ['단어','빈도']).sort_values('빈도',ascending=False).reset_index(drop=True)

In [7]:
tfd_ex_josa.head()

,단어,빈도
0,num,125607
1,하,88554
2,한,46390
3,및,35371
4,있,34588


### Wordcloud

In [10]:
from wordcloud import WordCloud #wordcloud 생성
from IPython.display import set_matplotlib_formats
import matplotlib.pyplot as plt

matplotlib.rc('font',family = 'Malgun Gothic')
set_matplotlib_formats('retina')
matplotlib.rc('axes',unicode_minus = False)
plt.rcParams["figure.figsize"] = (15,8)
wordcloud = WordCloud(font_path = 'C:/Windows/Fonts/malgun.ttf', background_color='white',colormap = "Accent_r",
            width=1500, height=1000).generate_from_frequencies(ex_josa_dic) #여기에 dic형태넣어주기

plt.imshow(wordcloud)
plt.axis('off')
plt.show;

NameError: name 'matplotlib' is not defined

## Document size check

In [11]:
print(df.shape)
print(tfd_tokens.shape)
print(tfd_nouns.shape)
print(tfd_ex_josa.shape) #어미와 조사 태그 제거 후 94개 token 감소

(1130, 9)
(14270, 2)
(9970, 2)
(14174, 2)


dataset은 10년동안 kospi200에 계속 편입되어있던 113개의 기업의 발간 사업보고서들인 1130개의 Documents입니다.
Mecab으로 형태소분석을 했을 때 총 토큰들과 그 중 명사만 뽑아 term document freqeuncy를 빈도가 높은 순 대로 살펴보았는데 각각 14270, 9970개의 단어 수를 기록했습니다. 형태소분석 단어집합들은 숫자를 치환한 'num'와 조사가 빈도수가 가장 많았습니다.

이중에서 조사와 어미의 pos tags를 제외한 토큰 수는 14174개로 96개가 감소했습니다.

다음으로는 tfd_nouns와 비교 후 이를 documnet filings 연도별로 매칭시켜 연도별 word counts의 frequency를 살펴보도록 하자.

In [12]:
df_count = df.copy()
df_count = df_count.loc[:,['연도','crp_nm','rpt_nm']]
word_count_ls = [len(word_count(x).items()) for x in df.str_tokens]
word_count_ex_josa = [len(word_count(x).items()) for x in df.str_ex_josa]
word_count_noun_ls = [len(word_count(x).items()) for x in df.str_nouns]
df_count['document_size'] = word_count_ls
df_count['document_ex_josa'] = word_count_ex_josa
df_count['document_noun'] = word_count_noun_ls

df_count.head()

,연도,crp_nm,rpt_nm,document_size,document_ex_josa,document_noun
0,2019,삼양홀딩스,사업보고서 (2018.12),502,474,352
1,2018,삼양홀딩스,사업보고서 (2017.12),509,486,353
2,2017,삼양홀딩스,사업보고서 (2016.12),395,374,268
3,2016,삼양홀딩스,사업보고서 (2015.12),402,379,284
4,2015,삼양홀딩스,사업보고서 (2014.12),419,393,289


In [13]:
df_count.document_size.describe()

count    1130.00000
mean      537.29646
std       264.90395
min        26.00000
25%       372.00000
50%       480.00000
75%       634.00000
max      1939.00000
Name: document_size, dtype: float64

토큰화를 했을 때 전체 토큰 기준으로 사업보고서의 document size에 대한 요약통계이다.

시계열적인 측면을 고려하지 않고 전체로 봤 을때 단어 수의 평균은 537개 , min 26 , max 1939이다.

In [14]:
mean_ls = [(df_count[df_count.연도 == x].document_size).mean() for x in df_count.연도.unique()]

In [15]:
df_count = df_count.sort_values(by = '연도')

year_noun_ls = [sum(df_count[df_count.연도 == x].document_noun) for x in df_count.연도.unique()]
year_ex_josa = [sum(df_count[df_count.연도 == x].document_ex_josa) for x in df_count.연도.unique()]
year_ls = [sum(df_count[df_count.연도 == x].document_size) for x in df_count.연도.unique()]

doc_size_by_year = pd.DataFrame(list(zip(df_count.연도.unique(),year_noun_ls,year_ex_josa,year_ls)),columns= ['year','doc_noun','doc_ex_josa','doc_total'])
doc_size_by_year = doc_size_by_year.sort_values(by = 'year', ascending= True).reset_index(drop=True)
doc_size_by_year = doc_size_by_year.set_index('year')

doc_size_by_year.iplot(kind = 'bar',title='연도별 총 단어집합의 개수',xTitle='year',yTitle='count')

In [16]:
df_count = df_count.sort_values(by = '연도')

mean_noun_ls = [(df_count[df_count.연도 == x].document_noun).mean() for x in df_count.연도.unique()]
mean_ex_josa = [(df_count[df_count.연도 == x].document_ex_josa).mean() for x in df_count.연도.unique()]
mean_ls = [(df_count[df_count.연도 == x].document_size).mean() for x in df_count.연도.unique()]

mean_size_by_year = pd.DataFrame(list(zip(df_count.연도.unique(),mean_noun_ls,mean_ex_josa,mean_ls)),columns= ['year','doc_noun','doc_ex_josa','doc_total'])
mean_size_by_year = mean_size_by_year.sort_values(by = 'year', ascending= True).reset_index(drop=True)
mean_size_by_year = mean_size_by_year.set_index('year')

mean_size_by_year.iplot(kind='bar', title ='연도별 보고서 내 평균단어의 개수',xTitle='year',yTitle='count')

Similarity_Cohen_Malloy_Nguyen(2019)Lazy Prices의 내용과 마찬가지로 미국 뿐만 아니라 한국에서도 10년간 사업보고서 내의 단어 빈도가 꾸준히 증가하였다. 토큰화된 사업보고서의 텍스트를 기준으로 하였으며 명사로 봤을 때도 단조 증가하는 양상은 동일하였다.

형태소 분석 이후 1.명사로만 토큰화, 2.조사와 어미를 제거하고 , 3.모든 형태소를 기준으로 해서 비교를 해보았다.

연도별로 총 단어집합을 count해서 보고 연도별로 개별 보고서 단어수의 평균으로 봤을 때도 거의 같은 모습이다.

## Document Similarity


Document의 유사도를 계산하기 위해서 논문에서 다루고 있는 Jaccard similarity 외에 Cosine similarity가 있다. 코사인 유사도는 두 벡터 간의 사잇각의 크기를 측정하여 유사도를 측정하는 방법이다. 텍스트마이닝에서는 각각의 document D1 , D2가 벡터로 표현되고.. D1의 term frequency만큼의 차원을 가진 벡터 V1으로 표현이 될 것이다. 연산을 위해서는 D1과 D2의 차원이 같아야 하므로 사업보고서의 단어수는 각각 다르기 떄문에 padding을 시켜줄 필요가 있다. 공식은 간단하게 분자는 각각의 벡터들의 내적, 분모는 두 벡터의 크기의 곱을 Euclidian distance로 구해주면 된다.

단순하게 단어 set의 교집합을 합집합으로 나누는 자카드 유사도보다는 문서의 의미론적인 유사도 파악에 있어서 이점이 있는지는 정확하게 모르겠지만 수치상으로 비교했을 때 코사인 유사도가 더 높은 것 같다.

텍스트마이닝에서 사이킷런의 패키지도 그렇고  TF-IDF를 가지고 코사인 유사도 계산을 하는데 TF-IDF란 TF(term frequency)에 IDF(inverse document frequency)를 곱한 것이다. 아이디어의 핵심은 단순히 단어의 빈도가 많다고 그 단어가 중요한 단어가 아니라 특정 문서(document)에서 많이 등장하는 단어인지 아니면 모든 문서에서 많이 등장하는 단어인지를 파악하여 weight를 줘야 한다는 것이다. 가령 한글의 조사 ‘은’이나 ‘가’는 형태소 토큰화를 했을때 빈번하게 등장하는 토큰일 것이지만 문서들간의 유사도를 따지는 데 있어서 중요한 단어라고 보기 어렵다. 
IDF는 N/df에 log를 씌워줘서 계산한다. 확률의 개념을 사용하여 원래는 N( 등장한 document의 총 수)를 실제 term이 등장한 document의 수인 df로 나눠준 것인데 이를 inverse하여 log를 씌웠다. 로그를 씌운 것은 곱셈계산을 더해서 할수있고 빈도가 가장 많이 나오는 단어들은 그렇지 않은 단어들에 비해 거의 2배씩 증가하면서 많아지기 때문에 이를 보정하기 위한 정규화 방법이라고 볼 수 있다.


In [17]:
def length_adjust(str_) : 
    return (2-len(str_)) * '0' + str_

def date_convertor(date) : 
    year = length_adjust(str(date.year))
    month = length_adjust(str(date.month))
    day = length_adjust(str(date.day))
    yyyymmdd = int(year+month+day)
    return yyyymmdd

In [18]:
def cal_sim(df,type_):
    
    
    naive_df = df.copy()
    lv2_index = pd.date_range(start='20100101', end='20191231')
    lv2_index = list(map(date_convertor,lv2_index))
    lv2_columns = naive_df.crp_nm.unique()

    lv2_df = \
    pd.DataFrame(np.full(shape=(len(lv2_index),len(lv2_columns)),fill_value=np.nan),index=lv2_index,columns=lv2_columns)
    
    for process,corp_name in enumerate(lv2_columns):
        print("Process : {} | Total : {}".format(process+1,len(lv2_columns)),end='\r')

        tmp_df = naive_df[naive_df.crp_nm == corp_name]

        min_yr = tmp_df['연도'].min() #2010
        max_yr = tmp_df['연도'].max() #2019

        tmp_tf = tfidf.fit_transform(tmp_df.str_tokens) #vectorize as matrix
        tmp_cosine = linear_kernel(tmp_tf,tmp_tf)

        sim_ls = []; year_ls = []

        for yr in range(min_yr,max_yr) :

            pre_df = tmp_df[tmp_df['연도'] == yr]
            next_df = tmp_df[tmp_df['연도'] == yr+1]

            publish_date = tmp_df[tmp_df['연도'] == yr+1]['rcp_dt']

            if len(pre_df) == 0 or len(next_df) ==0:
                continue

            else:
                if type_ == 'cosine':
                    concat_df = pd.concat([pre_df,next_df]) 
                    co_vec = tfidf.fit_transform(concat_df.str_nouns)
                    sim = linear_kernel(co_vec)[1][0]
                    
                elif type_ == 'jaccard':
                    pre_token = pre_df.str_tokens.map(lambda x : x.split(" ")).values
                    next_token = next_df.str_tokens.map(lambda x : x.split(" ")).values   
                    if type(pre_token[0]) == list and type(next_token[0]) == list :
                        sim = len(set(next_token[0])&set(pre_token[0]))/len(set(next_token[0] + pre_token[0]))
                    else : 
                        sim = len(set(next_token)&set(pre_token))/len(set(next_token + pre_token))
                    

            year_ls.append(publish_date.values[0])
            sim_ls.append(sim)

            assert len(year_ls) == len(sim_ls)
            lv2_df.loc[year_ls,corp_name] = sim_ls
    
    return lv2_df

In [19]:
%%time
cosine_df = cal_sim(df,'cosine')
jaccard_df = cal_sim(df,'jaccard')

Wall time: 10.8 sotal : 113


In [20]:
print(jaccard_df.mean().mean())
print(cosine_df.mean().mean())

0.6588759895058125
0.8494677428711447


In [21]:
cos_ls = []
jac_ls = []
for i in cosine_df.columns:
    for j in cosine_df.index:
        if cosine_df.loc[j,i] > 0:
            cos_ls.append(cosine_df.loc[j,i])
    
for i in jaccard_df.columns:
    for j in jaccard_df.index:
        if jaccard_df.loc[j,i] > 0:
           jac_ls.append(jaccard_df.loc[j,i])
        
sim_df = pd.DataFrame([jac_ls,cos_ls],index={'jaccard','cosine'})
sim_df = sim_df.T

sim_df.corr(method='pearson')

,cosine,jaccard
cosine,1.000000,0.795748
jaccard,0.795748,1.000000


In [24]:
np.where(cosine_df.index < 201

Int64Index([20100101, 20100102, 20100103, 20100104, 20100105, 20100106,
            20100107, 20100108, 20100109, 20100110,
            ...
            20191222, 20191223, 20191224, 20191225, 20191226, 20191227,
            20191228, 20191229, 20191230, 20191231],
           dtype='int64', length=3652)

In [22]:
cosine_df[cosine_df.loc[:,'삼양홀딩스'].isnull() != True]

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20110331,0.621127,NaN,0.771980,0.912223,0.841423,0.430739,0.924065,NaN,0.797388,0.806065,...,0.961784,NaN,NaN,0.748038,0.750325,NaN,0.792013,0.787497,0.934443,0.949902
20120330,0.436818,0.874424,0.798941,0.841013,0.804597,0.856951,0.692892,0.538199,0.702372,0.631874,...,NaN,NaN,0.788060,0.963774,0.771359,0.910826,0.788836,0.491008,0.802622,0.950659
20130401,0.774388,NaN,NaN,NaN,0.931310,0.628641,0.804912,0.842385,NaN,0.769667,...,0.878047,NaN,NaN,0.948461,0.789727,NaN,0.677898,0.786522,0.927470,NaN
20140331,0.873603,0.886677,0.831403,0.759648,0.923185,0.953634,0.927729,0.761984,0.981531,0.732650,...,0.893905,0.624440,0.884087,0.952865,0.892467,0.536256,0.685722,0.844429,0.936531,0.974787
20150331,0.891958,0.929742,0.879101,0.949373,0.936710,0.958416,0.938651,0.652661,0.231396,0.958864,...,0.890258,0.955313,0.620773,0.929024,0.839837,0.983116,0.943338,0.953741,0.888051,0.971678
20160330,0.899072,0.805286,0.818135,0.950721,0.902977,0.975679,0.966113,0.863341,0.516317,0.925270,...,0.917016,0.818009,0.956296,0.957298,0.739007,0.895230,0.888472,0.922541,0.726691,0.979432
20170331,0.833291,0.963686,0.821579,0.953268,0.906460,0.753047,0.924590,0.960381,0.782152,0.913321,...,0.934883,0.900208,0.970385,0.951501,0.826611,NaN,0.959097,0.973963,0.807702,0.971163
20180402,0.798196,0.878202,0.844026,NaN,NaN,0.921876,0.884828,0.927041,0.544283,0.934208,...,0.841492,0.813713,0.968287,0.966102,0.858144,NaN,0.865143,0.966008,NaN,0.966720
20190401,0.919505,0.791298,0.951103,NaN,0.695788,0.860791,0.898024,0.890701,NaN,0.869604,...,0.945114,0.898513,0.974699,0.988277,0.828114,0.903154,0.942776,0.989379,0.848119,0.972964


사업보고서의 유사도를 비교하기 위해 자카드 유사도와 코사인 유사도 두가지 방법으로 유사도를 측정하여 dataframe 형태로 저장하였다.

columns는 113개의 기업들, index는 2010년부터 2019년까지의 일별날짜로 사업보고서가 첫 번쨰로 filing된 날짜인 rcp_dt에 전년도와 다음년도 사업보고서의 유사도를 저장하였다. 만약 10년에 처음 보고서가 나오고 다음해인 11년 3월 31일에 다음 사업보고서가 나왔다면 20110331의 인덱스에 2010년과 2011년 보고서의 유사도값을 저장하였다. 연속된 해에 보고서가 한번이라도 없으면 이를 배제하였고 총 1000개의 유사도값이 나왔다.

자카드 유사도의 평균값은 0.66, 코사인 유사도의 평균값은 0.85이다. 코사인 유사도를 계산하는 공식에는 조금씩 차이가 있지만 보편적으로 쓰이는 사이킷런 패키지의  tf-idf 벡터화를 실행한 후 이들의 코사인 값을 계산했다.

자카드와 코사인 유사도로 계산한 값들의 correlation은 0.795748이다. 이 수치는 미국 시장에서의 두 방법론의 상관관계보다 높은 수치이다.


